In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm,logm
from scipy.spatial.transform import Rotation
import sympy

$\def\bm{\boldsymbol}$
# リー群の１種、SE(3)での座標変換に使ってみるリー代数
- 定数ベクトル$\bm{p}$に対しリー代数$X$をかける事で微分方程式を作れる
$$
\dot{\bm{p}}(t) = X\bm{p}(t)
$$
- これで速度がわかるが、単位時刻を掛けて移動したと考えれば、そのまま座標変換として使える

In [2]:
roll = 30 #deg
pitch = 20 #deg
yaw = 10 #deg

phi = np.array([np.deg2rad(roll), np.deg2rad(pitch), np.deg2rad(yaw)])
trans = np.array([1, 2, 3])

x = sympy.Symbol('x')
y = sympy.Symbol('y')
z = sympy.Symbol('z')
a = sympy.Symbol('a')
b = sympy.Symbol('b')
c = sympy.Symbol('c')

# SO(3)回転行列
Rx = sympy.Matrix(
    [[1, 0, 0],
     [0, sympy.cos(a), -sympy.sin(a)], 
     [0, sympy.sin(a), sympy.cos(a)],
    ]
)
Ry = sympy.Matrix(
    [[sympy.cos(b), 0, sympy.sin(b)],
     [0, 1, 0],
     [-sympy.sin(b), 0, sympy.cos(b)]
    ]
)
Rz = sympy.Matrix(
    [[sympy.cos(c), -sympy.sin(c), 0],
     [sympy.sin(c), sympy.cos(c), 0],
     [0, 0, 1]
    ]
)
print("ZYXオイラー:")
rot = Rz*Ry*Rx
display(sympy.trigsimp(rot))


#同時変換行列
T = sympy.Matrix([
    [*rot[0,:], x],
    [*rot[1,:], y],
    [*rot[2,:], z],
    [0, 0, 0, 1]])
print("同次変換行列:")
display(T)

T1 = T.subs(a, phi[0]).subs(b, phi[1]).subs(c, phi[2]).subs(x, trans[0]).subs(y, trans[1]).subs(z, trans[2])
# display(T1)
T1 = sympy.matrix2numpy(T1).astype(float)
rot1 = R.from_matrix(T1[:-1,:-1])

ZYXオイラー:


Matrix([
[cos(b)*cos(c), sin(a)*sin(b)*cos(c) - sin(c)*cos(a),  sin(a)*sin(c) + sin(b)*cos(a)*cos(c)],
[sin(c)*cos(b), sin(a)*sin(b)*sin(c) + cos(a)*cos(c), -sin(a)*cos(c) + sin(b)*sin(c)*cos(a)],
[      -sin(b),                        sin(a)*cos(b),                         cos(a)*cos(b)]])

同次変換行列:


Matrix([
[cos(b)*cos(c), sin(a)*sin(b)*cos(c) - sin(c)*cos(a),  sin(a)*sin(c) + sin(b)*cos(a)*cos(c), x],
[sin(c)*cos(b), sin(a)*sin(b)*sin(c) + cos(a)*cos(c), -sin(a)*cos(c) + sin(b)*sin(c)*cos(a), y],
[      -sin(b),                        sin(a)*cos(b),                         cos(a)*cos(b), z],
[            0,                                    0,                                     0, 1]])

$\def\bm{\boldsymbol}$

## ベクトル表示のパラメータ

パラメータ$\bm{\tau}=[\bm{\theta}~~\bm{\rho}]^T \in \mathbb{R}^6$は以下を満たす
$$
Exp([\bm{\tau]_\wedge})= 
\begin{bmatrix}
   Exp([\bm{\theta}]_\wedge]) & V(\bm{\theta})\bm{\rho}  \\
   \bm{0} & 1  
\end{bmatrix}\\
$$
ただし、$\bm{\theta}$は$R$を回転ベクトルに変換したものに等しい
また、$\theta=||\bm{\theta}||$とすると

$$
V(\bm{\theta})=\frac{\sin\theta}{\theta}I+\frac{1-\cos \theta}{\theta} \frac{[\bm{\theta}]_\wedge}{\theta}+ \left(1-\frac{\sin \theta}{\theta}\right)\frac{\bm{\theta}\bm{\theta}^T}{\theta^2}
$$
もしくは、式変形すると下のようにもなる（はず）
$$
V(\bm{\theta})=I+\frac{1-\cos \theta}{\theta} \frac{[\bm{\theta}]_\wedge}{\theta}+ \left(1-\frac{\sin \theta}{\theta}\right)\frac{[\bm{\theta}]_\wedge^2}{\theta^2}
$$

### 参考
- https://sterngerlach.github.io/lie-4.html
- https://arxiv.org/pdf/1812.01537.pdf

In [20]:
#基底を計算
K = sympy.diff(T, a).subs(a, 0).subs(b, 0).subs(c, 0)
L = sympy.diff(T, b).subs(a, 0).subs(b, 0).subs(c, 0)
M = sympy.diff(T, c).subs(a, 0).subs(b, 0).subs(c, 0)
N = sympy.diff(T, x)
O = sympy.diff(T, y)
P = sympy.diff(T, z)
print("bases:")
display(K,L,M,N,O,P)

K = sympy.matrix2numpy(K).astype(float)
L = sympy.matrix2numpy(L).astype(float)
M = sympy.matrix2numpy(M).astype(float)
N = sympy.matrix2numpy(N).astype(float)
O = sympy.matrix2numpy(O).astype(float)
P = sympy.matrix2numpy(P).astype(float)

# リー代数を計算
theta = rot1.as_rotvec() #回転ベクトルはズルしてsicpy使って変換する…
theta_norm = np.linalg.norm(theta)
tmp = theta[0] * K + theta[1] * L + theta[2] * M
V1 =  np.sin(theta_norm)/theta_norm * np.eye(3) + ((1 - np.cos(theta_norm))/theta_norm**2) * tmp[:-1,:-1] + (1 - np.sin(theta_norm)/theta_norm) * theta.reshape(3,1) @ theta.reshape(1,3) * (1/theta_norm**2)
# V1 =  np.eye(3) + ((1 - np.cos(theta_norm))/theta_norm**2) * tmp[:-1,:-1] + (1 - np.sin(theta_norm)/theta_norm) * tmp[:-1,:-1] @ tmp[:-1,:-1] * (1/theta_norm**2) #こっちでも結果は同じ
rho1 = np.linalg.inv(V1) @ trans
alg1 = tmp + rho1[0] * N + rho1[1] * O + rho1[2] * P

print("Lie algebra:\n{}".format(alg1))

bases:


Matrix([
[0, 0,  0, 0],
[0, 0, -1, 0],
[0, 1,  0, 0],
[0, 0,  0, 0]])

Matrix([
[ 0, 0, 1, 0],
[ 0, 0, 0, 0],
[-1, 0, 0, 0],
[ 0, 0, 0, 0]])

Matrix([
[0, -1, 0, 0],
[1,  0, 0, 0],
[0,  0, 0, 0],
[0,  0, 0, 0]])

Matrix([
[0, 0, 0, 1],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 1],
[0, 0, 0, 0],
[0, 0, 0, 0]])

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 1],
[0, 0, 0, 0]])

Lie algebra:
[[ 0.         -0.07752532  0.38485157  0.52821958]
 [ 0.07752532  0.         -0.48647923  2.67345732]
 [-0.38485157  0.48647923  0.          2.61728981]
 [ 0.          0.          0.          0.        ]]


In [16]:
T1.astype(np.float16)

array([[ 0.9253 ,  0.01804,  0.3784 ,  1.     ],
       [ 0.1632 ,  0.8823 , -0.441  ,  2.     ],
       [-0.342  ,  0.4697 ,  0.814  ,  3.     ],
       [ 0.     ,  0.     ,  0.     ,  1.     ]], dtype=float16)

In [17]:
expm(alg1).astype(np.float16)

array([[ 0.9253 ,  0.01804,  0.3784 ,  1.     ],
       [ 0.1632 ,  0.8823 , -0.441  ,  2.     ],
       [-0.342  ,  0.4697 ,  0.814  ,  3.     ],
       [-0.     ,  0.     ,  0.     ,  1.     ]], dtype=float16)

一致した!!